In [ ]:
import pandas as pd
import plotly.express as px

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

In [ ]:
app = dash.Dash(__name__)

In [ ]:
# Import data
df = pd.read_csv(r"C:\NOT_C\Coursera\IBM\capstone\youtube_dash\intro_bees.csv")
# df.head()

In [ ]:
# Clean data
df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace = True)

# df.head()

In [ ]:
# App layout
bee_killers = ["Disease", "Other", "Pesticides", "Pests_excl_Varroa", "Unknown", "Varroa_mites"]

app.layout = html.Div([
    
    html.H1("Visualizing Affected Bee Colonies using Dashboards with Dash", style={'text-align':'center'}),
    
    html.H3(" 1) % of bee colonies by State: Choropleth graph and Bar graph", style={'text-align':'left'}),
    
    dcc.Dropdown(
        id='select_year',
        options=[
            {'label':'2015', 'value':2015},
            {'label':'2016', 'value':2016}, # value should be represented the same way as in the source (df, db etc)
            {'label':'2017', 'value':2017}, # here the dtype of value is int64 (same as in df)
            {'label':'2018', 'value':2018}],
        multi=False,
        value=2015,
        style={'width': '40%'}
    ),
    
    html.Div(id='output_container', children=[], style={'text-align':'right'}),
    html.Br(),
    
    dcc.Graph(id='my_bee_map', figure={}),
    
    dcc.Graph(id='my_bee_bar', figure={}),
    html.Br(),
    
    html.H3(" 2) Affect of various bee-killer variables on bee colonies: Line Graph", {'text-align':'left'}),
    dcc.Dropdown(
        id='affected_by',
        options=[{'label':x, 'value':x} for x in bee_killers],
        multi=False,
        value='Varroa_mites',
        style={'width': '40%'}
    ),
    html.Br(),
    dcc.Graph(id='my_bee_line', figure={})
    
])

In [ ]:
# Connect plotly graphs with dash components

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure'),
     Output(component_id='my_bee_bar', component_property='figure'),
     Output(component_id='my_bee_line', component_property='figure')],
    [Input(component_id= 'select_year', component_property='value'),
     Input(component_id= 'affected_by', component_property='value')]
)

def update_graph(selected_year, affected_by): # the input: component_property goes here
    print(selected_year)
    print(type(selected_year))
    
    container= 'The year chosen by user was: {}'.format(selected_year)
    
    # filter the df 
    dff = df.copy()              # always use a copy to process data inside the fn.
    dff = dff[dff['Year'] == selected_year]
    dff = dff[dff['Affected by'] == 'Varroa_mites']
    
    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State','Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted':'% of Bee Colonies'},
        template='plotly_dark'
    )
    
    fig2 = px.bar(
        data_frame=dff,
        x=dff['State'],
        y=dff['Pct of Colonies Impacted'],
        hover_data=['State','Pct of Colonies Impacted'],
        labels={'Pct of Colonies Impacted':'% of Bee Colonies'},
        orientation='v',
        template='plotly_dark'
    )
    dff2 = df.copy()
    dff2 = dff2[(dff2['State'] == 'Texas') | (dff2['State'] == 'New Mexico') | (dff2['State'] == 'New York')]
    dff2 = dff2[dff2['Affected by'] == affected_by]
    
    fig3 = px.line(
        data_frame=dff2,
        x=dff2['Year'].astype('str'),
        y=dff2['Pct of Colonies Impacted'],
        color=dff2['State'],
        labels={'Pct of Colonies Impacted':'% of Bee Colonies'},
        template='plotly_dark'
    )
    
    return container, fig, fig2, fig3   # return the output(s)

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)
    